In [5]:
from langchain_google_genai import GoogleGenerativeAI
llm=GoogleGenerativeAI(model="gemini-1.5-flash", max_output_tokens=1000)

### PromptsTemplate in langchain <br> Templates with Placeholders

In [ ]:
from langchain_core.prompts import PromptTemplate

tempalte="hey my name is {name} and I am {age} years old"
prompt=PromptTemplate.from_template(tempalte)

print("prompt type:", type(prompt))
print("inptut variables:", prompt.input_variables)
print(prompt.format(name="james", age=22))

prompt type: <class 'langchain_core.prompts.prompt.PromptTemplate'>
inptut variables: ['age', 'name']
hey my name is ujwal and I am 22 years old


### Partial Prompts to define some Default var

In [ ]:
tempalte="hey my name is {name} and I am {age} years old and I am a {profession}"
prompt=PromptTemplate.from_template(tempalte)

prompt_with_default=prompt.partial(name="james", age=22)

#now provide only the profession
print(prompt_with_default.format(profession="developer"))

hey my name is ujwal and I am 22 years old and I am a developer


### Template with the multiple messages of CHAT models

In [3]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

chat_tempalte = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    HumanMessagePromptTemplate.from_template("{question}"),
])
chat=chat_tempalte.partial(question="What is LangChain?")
print(chat.format())

System: You are a helpful assistant
Human: What is LangChain?


### Few Shot prompting

In [4]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

exmaple=[
    {"input": "What is LangChain?", "output": "LangChain is a framework for building applications with LLMs."},
    {"input": "What is LangGraph?", "output": "LangGraph is a framework"}
]

example_prompt=ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("{input}"),
    AIMessagePromptTemplate.from_template("{output}")
    ])
few_shot_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=exmaple,
)
final_prompt=ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    few_shot_prompt,
    HumanMessagePromptTemplate.from_template("{input}")
])
print(final_prompt.format(input="What is LangGraph?"))

System: You are a helpful assistant
Human: What is LangChain?
AI: LangChain is a framework for building applications with LLMs.
Human: What is LangGraph?
AI: LangGraph is a framework
Human: What is LangGraph?


# Output Parser

### Chains fro using | for chains the prompt must be a Template

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt="what is the joke about {animal}?"
chat_promt=ChatPromptTemplate.from_template(prompt)
chain_no_parser= chat_promt | llm
response= chain_no_parser.invoke(chat_promt.format(animal="lion"))
print("Response without parser:", response)

Response without parser: There are many jokes about lions!  To tell you a good one, I need a little more information. What *kind* of joke are you looking for?  For example:

* **A pun?** (e.g.,  What do you call a lazy kangaroo? Pouch potato!  A lion joke could play on words like "mane" or "roar".)
* **A one-liner?** (A short, simple joke.)
* **A longer story joke?**
* **A joke about a lion's personality?** (e.g.,  a lion being grumpy, regal, etc.)

Tell me what style of joke you prefer, and I'll try my best!


In [19]:
chain_parser= chat_promt | llm | StrOutputParser()
response_with_parser= chain_parser.invoke({"animal": "lion"})
print("Response with parser:", response_with_parser)

Response with parser: There are many lion jokes!  To tell you one, I need a little more information.  What *kind* of lion joke are you looking for?  For example, are you looking for:

* **A pun?** (e.g.,  playing on words related to lions)
* **A one-liner?** (a short, quick joke)
* **A longer story joke?**
* **A joke about a specific aspect of lions?** (e.g., their mane, their roar)


Tell me what you'd prefer and I'll do my best!


## JSON parser

In [21]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
format_instructions=json_parser.get_format_instructions()
print("Format instructions for JSON parser:", format_instructions)

Format instructions for JSON parser: Return a JSON object.


In [28]:
prompt=ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    HumanMessagePromptTemplate.from_template("extract the name and age from the following text: {text}, {format_instructions}"),
]).partial(
    format_instructions=format_instructions
)

chain_json_parser = prompt | llm | json_parser
text="My name is james and I am 22 years old."
response_json_parser = chain_json_parser.invoke({"text": text})
print("Response with JSON parser:", response_json_parser)
print(response_json_parser["name"])  # Accessing the name 

Response with JSON parser: {'name': 'james', 'age': 22}
james


## Pydantic Parser<br>for more deailed

In [35]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    animal: str = Field(description="The animal the joke is about")
    joke: str = Field(description="The joke itself")
from langchain_core.output_parsers import PydanticOutputParser
pydantic_parser = PydanticOutputParser(pydantic_object=Joke)

format_instructions = pydantic_parser.get_format_instructions()
print("Format instructions for Pydantic parser:", format_instructions)

promt=ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    HumanMessagePromptTemplate.from_template("tell me a joke about {animal}, {format_instructions}"),
]).partial(format_instructions=format_instructions)

chain = promt | llm | pydantic_parser
response = chain.invoke({"animal": "lion"})
print("Response with Pydantic parser:", response)
print("Joke:", response.joke)  # Accessing the joke
print("Animal:", response.animal)  # Accessing the animal

Format instructions for Pydantic parser: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"animal": {"description": "The animal the joke is about", "title": "Animal", "type": "string"}, "joke": {"description": "The joke itself", "title": "Joke", "type": "string"}}, "required": ["animal", "joke"]}
```
Response with Pydantic parser: animal='Lion' joke='Why are lions bad dancers? Because they have no mane-ners!'
Joke: Why are lions bad dancers? Because they have no mane-ners!
Animal: Lion
